## NBA Basketball - Offensive and Defensive Game Play

In [1]:
from pymongo import MongoClient
import pprint

import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns

# Requests sends and receives HTTP requests.
import requests

import json
import time
import copy

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
from nba_api.stats.static import teams
from nba_api.stats.static import players
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.endpoints import leaguedashteamstats
from nba_api.stats.endpoints import boxscorefourfactorsv2
from nba_api.stats.endpoints import boxscoreadvancedv2

In [4]:
teams_df = pd.DataFrame(teams.get_teams())
teams_df.head()

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


In [5]:
player_df = pd.DataFrame(players.get_players())
player_df.head()

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False


In [6]:
#game_log_1819 = leaguegamelog.LeagueGameLog(counter = 0, direction = "DESC",
#                 league_id = "00", player_or_team_abbreviation = "T", season = "2018-19",
#                 season_type_all_star = "Regular Season", sorter = "DATE").get_data_frames()

In [7]:
#game_log_1819[0].to_csv(r'../data_1/game_log_1819.csv', index=False)

In [8]:
types_dict = {'GAME_ID': str}
game_log_1819 = pd.read_csv('../data_1/game_log_1819.csv', dtype=types_dict)
game_log_1819.head() 
#2018-19 Regular season game stats 

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22018,1610612744,GSW,Golden State Warriors,0021801225,2019-04-10,GSW @ MEM,L,240,46,92,0.500,13,30,0.433,12,14,0.857,9,30,39,32,3,5,14,17,117,-15,1
1,22018,1610612763,MEM,Memphis Grizzlies,0021801225,2019-04-10,MEM vs. GSW,W,240,48,98,0.490,21,46,0.457,15,18,0.833,15,34,49,25,9,0,11,18,132,15,1
2,22018,1610612766,CHA,Charlotte Hornets,0021801222,2019-04-10,CHA vs. ORL,L,240,41,78,0.526,8,29,0.276,24,27,0.889,4,28,32,21,3,4,8,14,114,-8,1
3,22018,1610612753,ORL,Orlando Magic,0021801222,2019-04-10,ORL @ CHA,W,240,48,88,0.545,11,30,0.367,15,20,0.750,9,30,39,24,5,2,6,19,122,8,1
4,22018,1610612750,MIN,Minnesota Timberwolves,0021801228,2019-04-10,MIN @ DEN,L,240,39,91,0.429,13,32,0.406,4,6,0.667,7,34,41,24,6,0,10,22,95,-4,1


In [9]:
game_id_lst = game_log_1819['GAME_ID'].tolist() #list created to iterate over advanced game stats below

In [10]:
#leaguedashteamstats = leaguedashteamstats.LeagueDashTeamStats(season = "2018-19", season_type_all_star='Regular Season').get_data_frames()

In [11]:
#leaguedashteamstats[0].to_csv(r'../data_1/leaguedashteamstats.csv', index=False)

In [12]:
leaguedashteamstats = pd.read_csv('../data_1/leaguedashteamstats.csv')
leaguedashteamstats.head()
#league stats by team for 2018-19 regular season

,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,CFID,CFPARAMS
0,1610612737,Atlanta Hawks,82,29,53,0.354,3971.0,3392,7524,0.451,1067,3034,0.352,1443,1918,0.752,955,2825,3780,2118,1397.0,675,419,448,1932,1817,9294,-494.0,1,26,26,26,7,15,4,21,4,3,16,18,12,21,4,18,12,10,30,11,14,24,29,5,12,26,10,Atlanta Hawks
1,1610612738,Boston Celtics,82,49,33,0.598,3956.0,3451,7423,0.465,1032,2829,0.365,1282,1598,0.802,804,2849,3653,2155,1052.0,706,435,317,1670,1602,9216,364.0,1,9,9,9,18,11,9,14,6,7,7,28,29,4,20,16,22,5,3,4,10,2,12,26,14,6,10,Boston Celtics
2,1610612751,Brooklyn Nets,82,42,40,0.512,3996.0,3301,7358,0.449,1047,2965,0.353,1555,2088,0.745,900,2919,3819,1954,1236.0,539,339,432,1763,1807,9204,-6.0,1,14,14,14,1,20,14,25,5,5,14,6,4,24,10,10,7,21,26,27,28,22,23,7,15,15,10,Brooklyn Nets
3,1610612766,Charlotte Hornets,82,39,43,0.476,3966.0,3297,7362,0.448,977,2783,0.351,1510,1895,0.797,814,2778,3592,1905,1001.0,591,405,492,1550,1686,9081,-90.0,1,17,17,17,11,21,12,26,11,10,18,9,16,6,19,22,23,24,2,21,18,29,3,18,19,18,10,Charlotte Hornets
4,1610612741,Chicago Bulls,82,22,60,0.268,3981.0,3266,7205,0.453,745,2123,0.351,1328,1695,0.783,718,2799,3517,1796,1159.0,603,351,473,1663,1534,8605,-690.0,1,27,27,27,3,23,22,20,30,27,19,25,26,11,30,21,25,27,19,20,26,28,9,30,27,27,10,Chicago Bulls


In [13]:
#boxscorefourfactor = boxscorefourfactorsv2.BoxScoreFourFactorsV2(game_id='0021800997').get_data_frames()

In [14]:
#boxscorefourfactor[0].to_csv(r'../data_1/boxscorefourfactor.csv', index=False)

In [15]:
boxscorefourfactor = pd.read_csv('../data_1/boxscorefourfactor.csv', dtype=types_dict)
boxscorefourfactor.head()
#box score data by players in game DAL v. HOU

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,EFG_PCT,FTA_RATE,TM_TOV_PCT,OREB_PCT,OPP_EFG_PCT,OPP_FTA_RATE,OPP_TOV_PCT,OPP_OREB_PCT
0,0021800997,1610612745,HOU,Houston,201569,Eric Gordon,F,NaN,31:48,0.432,0.220,0.099,0.286,0.459,0.224,0.214,0.261
1,0021800997,1610612745,HOU,Houston,200782,P.J. Tucker,F,NaN,37:51,0.471,0.286,0.141,0.293,0.441,0.339,0.206,0.345
2,0021800997,1610612745,HOU,Houston,203991,Clint Capela,C,NaN,35:39,0.409,0.242,0.155,0.302,0.464,0.321,0.197,0.280
3,0021800997,1610612745,HOU,Houston,201935,James Harden,G,NaN,33:55,0.467,0.328,0.143,0.278,0.510,0.294,0.220,0.320
4,0021800997,1610612745,HOU,Houston,101108,Chris Paul,G,NaN,32:50,0.452,0.242,0.148,0.282,0.490,0.353,0.194,0.217


In [16]:
# boxscoreadvancedv2_df = pd.DataFrame()
# for game_id in game_id_lst:
#     a = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=game_id).team_stats
#     b = a.get_data_frame()
#     boxscoreadvancedv2_df = boxscoreadvancedv2_df.append(b, ignore_index=True)
#     time.sleep(2)

In [17]:
# boxscoreadvancedv2_df = boxscoreadvancedv2_df.drop_duplicates(ignore_index=True)

In [18]:
# boxscoreadvancedv2_df.to_csv(r'../data_1/boxscoreadvancedv2.csv', index=False)

In [19]:
boxscoreadvancedv2 = pd.read_csv('../data_1/boxscoreadvancedv2.csv')
boxscoreadvancedv2.head()

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,E_NET_RATING,NET_RATING,AST_PCT,AST_TOV,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,E_TM_TOV_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE
0,21801225,1610612763,Grizzlies,MEM,Memphis,240:00,129.5,130.7,113.4,115.8,16.1,14.9,0.521,2.27,17.6,0.346,0.745,0.535,10.793,10.9,0.597,0.623,1.0,0.200,102.54,101.0,84.17,101,0.533
1,21801225,1610612744,Warriors,GSW,Golden State,240:00,113.4,115.8,129.5,130.7,-16.1,-14.9,0.696,2.29,22.2,0.255,0.654,0.465,13.571,13.9,0.571,0.596,1.0,0.198,102.54,101.0,84.17,101,0.467
2,21801222,1610612753,Magic,ORL,Orlando,240:00,130.1,131.2,121.4,122.6,8.6,8.6,0.500,4.00,18.9,0.286,0.816,0.538,6.397,6.5,0.608,0.630,1.0,0.200,93.84,93.0,77.50,93,0.519
3,21801222,1610612766,Hornets,CHA,Charlotte,240:00,121.4,122.6,130.1,131.2,-8.6,-8.6,0.512,2.63,17.7,0.184,0.714,0.463,8.522,8.6,0.577,0.634,1.0,0.200,93.84,93.0,77.50,93,0.481
4,21801228,1610612750,Timberwolves,MIN,Minnesota,240:00,98.3,99.0,103.7,103.1,-5.4,-4.2,0.615,2.40,18.8,0.167,0.745,0.448,10.348,10.4,0.500,0.507,1.0,0.199,96.06,96.0,80.00,96,0.438


### General Dataset Information

In [ ]:
game_log_1819.shape

In [ ]:
game_log_1819.info()

In [ ]:
game_log_1819.describe()

In [ ]:
leaguedashteamstats.shape

In [ ]:
leaguedashteamstats.info()

In [ ]:
leaguedashteamstats.describe()

In [ ]:
boxscorefourfactor.shape

In [ ]:
boxscorefourfactor.info()

In [ ]:
boxscorefourfactor.describe()

### Dataset Preparation


### Exploratory Data Analysis

In [ ]:
game_log_1819.head()

In [ ]:
#game_groupby_team = game_filter_wins.groupby(
#    ['TEAM_ABBREVIATION']).agg({'WL': 'count'})
#game_groupby_team

In [ ]:
game_win_query = game_log_1819.query('WL == "W"')
fig, ax = plt.subplots(figsize=(16,12))
sns.countplot(x='TEAM_ABBREVIATION', data=game_win_query, ax=ax);

In [ ]:
game_loss_query = game_log_1819.query('WL == "L"')
fig, ax = plt.subplots(figsize=(16,12))
sns.countplot(x='TEAM_ABBREVIATION', data=game_loss_query, ax=ax);

In [ ]:
fig, ax = plt.subplots()
ax.hist(game_win_query['FG_PCT'], alpha=0.5, bins=35, label='Winning Teams')
ax.hist(game_loss_query['FG_PCT'], alpha=0.5, bins=35, label='Losing Teams')
ax.set_xlabel('Field Goal Percentage', size=12)
ax.set_ylabel('Frequency of Field Goal Percentage', size=12)
ax.set_title('Field Goal Game Percentage Histogram', size=15)
ax.legend(prop={'size': 10});

In [ ]:
fig, ax = plt.subplots()
ax.hist(game_win_query['FG3_PCT'], alpha=0.5, bins=35, label='Winning Teams')
ax.hist(game_loss_query['FG3_PCT'], alpha=0.5, bins=35, label='Losing Teams')
ax.set_xlabel('3-Point Field Goal Percentage', size=12)
ax.set_ylabel('Frequency of 3-Point Field Goal Percentage', size=12)
ax.set_title('3-Point Field Goal Game Percentage Histogram', size=15)
ax.legend(prop={'size': 10});

In [ ]:
fig, ax = plt.subplots()
ax.hist(game_win_query['FT_PCT'], alpha=0.5, bins=35, label='Winning Teams')
ax.hist(game_loss_query['FT_PCT'], alpha=0.5, bins=35, label='Losing Teams')
ax.set_xlabel('Free Throw Percentage', size=12)
ax.set_ylabel('Frequency of Free Throw Percentage', size=12)
ax.set_title('Free Throw Game Percentage Histogram', size=15)
ax.legend(prop={'size': 10});

In [ ]:
fig, ax = plt.subplots()
ax.hist(game_win_query['REB'], alpha=0.5, bins=35, label='Winning Teams')
ax.hist(game_loss_query['REB'], alpha=0.5, bins=35, label='Losing Teams')
ax.set_xlabel('Number of Game Rebounds', size=12)
ax.set_ylabel('Frequency of Number of Game Rebounds', size=12)
ax.set_title('Number of Game Rebounds Histogram', size=15)
ax.legend(prop={'size': 10});

In [ ]:
fig, ax = plt.subplots()
ax.hist(game_win_query['OREB'], alpha=0.5, bins=35, label='Winning Teams')
ax.hist(game_loss_query['OREB'], alpha=0.5, bins=35, label='Losing Teams')
ax.set_xlabel('Number of Offensive Rebounds', size=12)
ax.set_ylabel('Frequency of Number of Offensive Rebounds', size=12)
ax.set_title('Number of Offensive Game Rebounds Histogram', size=15)
ax.legend(prop={'size': 10});

In [ ]:
fig, ax = plt.subplots()
ax.hist(game_win_query['DREB'], alpha=0.5, bins=35, label='Winning Teams')
ax.hist(game_loss_query['DREB'], alpha=0.5, bins=35, label='Losing Teams');
ax.set_xlabel('Number of Defensive Rebounds', size=12)
ax.set_ylabel('Frequency of Number of Defensive Rebounds', size=12)
ax.set_title('Number of Defensive Game Rebounds Histogram', size=15)
ax.legend(prop={'size': 10});

In [ ]:
fig, ax = plt.subplots()
ax.hist(game_win_query['AST'], alpha=0.5, bins=35, label='Winning Teams')
ax.hist(game_loss_query['AST'], alpha=0.5, bins=35, label='Losing Teams')
ax.set_xlabel('Number of Assists', size=12)
ax.set_ylabel('Frequency of Number of Assists', size=12)
ax.set_title('Number of Game Assists Histogram', size=15)
ax.legend(prop={'size': 10});

In [ ]:
fig, ax = plt.subplots()
ax.hist(game_win_query['STL'], alpha=0.5, bins=35, label='Winning Teams')
ax.hist(game_loss_query['STL'], alpha=0.5, bins=35, label='Losing Teams')
ax.set_xlabel('Number of Steals', size=12)
ax.set_ylabel('Frequency of Number of Steals', size=12)
ax.set_title('Number of Game Steals Histogram', size=15)
ax.legend(prop={'size': 10});

In [ ]:
fig, ax = plt.subplots()
ax.hist(game_win_query['BLK'], alpha=0.5, bins=35, label='Winning Teams')
ax.hist(game_loss_query['BLK'], alpha=0.5, bins=35, label='Losing Teams')
ax.set_xlabel('Number of Blocks', size=12)
ax.set_ylabel('Frequency of Number of Blocks', size=12)
ax.set_title('Number of Game Blocks Histogram', size=15)
ax.legend(prop={'size': 10});

In [ ]:
fig, ax = plt.subplots()
ax.hist(game_win_query['TOV'], alpha=0.5, bins=35, label='Winning Teams')
ax.hist(game_loss_query['TOV'], alpha=0.5, bins=35, label='Losing Teams')
ax.set_xlabel('Number of Turnovers', size=12)
ax.set_ylabel('Frequency of Number of Turnovers', size=12)
ax.set_title('Number of Game Turnovers Histogram', size=15)
ax.legend(prop={'size': 10});

fig, ax = plt.subplots()
ax.hist(game_log_1819['FGA'], bins=50)
plt.show()

fig, ax = plt.subplots()
ax.hist(game_log_1819['FGM'], bins=50)
plt.show()

fig, ax = plt.subplots()
ax.hist(game_log_1819['FG_PCT'], bins=50)
plt.show()

fig, ax = plt.subplots()
ax.scatter(game_log_1819['FGA'], game_log_1819['FGM'])
ax.set_xlabel('Field Goals Attempted')
ax.set_ylabel('Field Goals Made')
plt.show()

fig, ax = plt.subplots()
ax.scatter(game_log_1819['FG3A'], game_log_1819['FG3M'])
ax.set_xlabel('3-Point Field Goals Attempted')
ax.set_ylabel('3-Point Field Goals Made')
plt.show()

fig, ax = plt.subplots()
ax.scatter(game_log_1819['OREB'], game_log_1819['DREB'])
ax.set_xlabel('Offensive Rebounds')
ax.set_ylabel('Defensive Rebounds')
plt.show()

### Hypothesis Tests

**Scientific question:** Is there a play between offensive and defensive play that is more effective in achieving a game win?

**Null:** There is no difference between the impact of offensive or defensive play in achieving a game win.

**Alternative:** There is a difference between the impact of offensive or defensive play in achieving a game win.

**Test statistic:** 2,460 regular season games from the 20xx - xx NBA season
(=82 games per team * 30 NBA teams)

metrics to test:
offensive play
    -home effective fg%
    -oppo turnover %
    -home offensive rebound %
    -home free throw rate
defensive play
    -oppo effective fg%
    -home turnover %
    -home defensive rebound %
    -oppo free throw rate

In [ ]:
def welch_test_statistic(sample_1, sample_2):
    numerator = np.mean(sample_1) - np.mean(sample_2)
    denominator_sq = (np.var(sample_1) / len(sample_1)) + \
                        (np.var(sample_2) / len(sample_2))
    return numerator / np.sqrt(denominator_sq)

In [ ]:
def welch_satterhwaithe_df(sample_1, sample_2):
    ss1 = len(sample_1)
    ss2 = len(sample_2)
    df = (
        ((np.var(sample_1)/ss1 + np.var(sample_2)/ss2)**(2.0)) / 
        ((np.var(sample_1)/ss1)**(2.0)/(ss1 - 1) + (np.var(sample_2)/ss2)**(2.0)/(ss2 - 1))
    )
    return df

#### 3-Point Field Goal Percentages

In [ ]:
game_win_query['FG3_PCT']

In [ ]:
FG3_PCT_df = welch_satterhwaithe_df(list(game_win_query['FG3_PCT']), list(game_loss_query['FG3_PCT']))
FG3_PCT_df

In [ ]:
FG3_PCT_dist = stats.t(FG3_PCT_df)
FG3_PCT_dist

In [ ]:
FG3_PCT_test_stat = welch_test_statistic(game_win_query['FG3_PCT'], game_loss_query['FG3_PCT'])
FG3_PCT_test_stat

In [ ]:
np.mean(game_win_query['FG3_PCT']) - np.mean(game_loss_query['FG3_PCT'])

In [ ]:
np.sqrt((np.var(game_win_query['FG3_PCT']) / len(game_win_query['FG3_PCT'])) + (np.var(game_loss_query['FG3_PCT']) / len(game_loss_query['FG3_PCT'])))

In [ ]:
# p-val
x = np.linspace(-3, 3, num=250)

fig, ax = plt.subplots(1, figsize=(16, 3))
FG3_PCT_dist = stats.t(FG3_PCT_df)
ax.plot(x, FG3_PCT_dist.pdf(x), linewidth=2, label="Degree of Freedom: {:2.2f}".format(FG3_PCT_df))
_ = ax.fill_between(x, FG3_PCT_dist.pdf(x), where=(x >= -FG3_PCT_test_stat), color="red", alpha=0.25)
_ = ax.fill_between(x, FG3_PCT_dist.pdf(x), where=(x <= FG3_PCT_test_stat), color="red", alpha=0.25)
ax.legend()
ax.set_title("p-value Region");

In [ ]:
FG3_PCT_p_value = FG3_PCT_dist.cdf(FG3_PCT_test_stat) + (1 - FG3_PCT_dist.cdf(-FG3_PCT_test_stat))
FG3_PCT_p_value

In [ ]:
FG3_PCT_dist.cdf(-FG3_PCT_test_stat)

In [ ]:
FG3_PCT_dist = stats.t(FG3_PCT_df).cdf()

In [ ]:
stats.ttest_ind(game_win_query['FG3_PCT'], game_loss_query['FG3_PCT'], equal_var=False)

In [ ]:
game_win_query['FG3_PCT'].mean()

#### Free Goal Attempts

In [ ]:
FGA_df = welch_satterhwaithe_df(list(game_win_query['FGA']), list(game_loss_query['FGA']))
FGA_df

In [ ]:
FGA_dist = stats.t(FGA_df)
FGA_dist

In [ ]:
FGA_test_stat = welch_test_statistic(game_win_query['FGA'], game_loss_query['FGA'])
FGA_test_stat

In [ ]:
# p-val
x = np.linspace(-3, 3, num=250)

fig, ax = plt.subplots(1, figsize=(16, 3))
FGA_dist = stats.t(FGA_df)
ax.plot(x, FGA_dist.pdf(x), linewidth=2, label="Degree of Freedom: {:2.2f}".format(FGA_df))
_ = ax.fill_between(x, FGA_dist.pdf(x), where=(x >= -FGA_test_stat), color="red", alpha=0.25)
_ = ax.fill_between(x, FGA_dist.pdf(x), where=(x <= FGA_test_stat), color="red", alpha=0.25)
ax.legend()
ax.set_title("p-value Region");

In [ ]:
FGA_p_value = FGA_dist.cdf(FGA_test_stat) + (1 - FGA_dist.cdf(-FGA_test_stat))
FGA_p_value

Dataset: using proxies, such as either offensive (ORtg) and defensive (DRtg) ratings or offense and defense four factors, for efficient offensive and defensive play respectively

### Conclusion

Bear in mind what the four factors actually measure e.g. for instance to clarify, the FT rate is Free Throws Attempted divided by Field Goals Attempted - 

"the biggest aspect of ‘free throws’ is actually attempting them, not making them. Teams that get to the line more are more effective than teams that make a higher percentage of their free throws…"